In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import config
from datasets.datasethandler import DatasetHandler
datasetHandler = DatasetHandler()

# Tutorial: Hyperparameter Search

In [3]:
import torch
import pytorch_lightning as pl
import optuna
from optuna.integration import PyTorchLightningPruningCallback

from classification.models.M5 import M5PLModule
from classification.models.SpectrogramCNN import SpectrogramCNNPLModule
from classification.models.DeepRecursiveCNN import DeepRecursiveCNNPLModule
#from classification.models.MelSpectrogramCNN_8K import MelSpectrogramCNN_8KPLModule
from classification.models.DeepRecursiveCNN8k import DeepRecursiveCNN8kPLModule

from classification.models.SpectrogramCNN_8K_Dataset2 import SpectrogramCNN_8K_Dataset2_PLModule
from classification.models.CRNN_8k_D2 import CRNN8k_D2_PLModule
from classification.trainer.HyperParamSearch import MetricsCallback, save_model

In [4]:
from pytorch_lightning.callbacks import Callback

class SaveBestCallback(Callback): 
    def __init__(self, model_name = "newest_model", add_v_number = True):
        super().__init__()
        self.model_name = model_name
        self.best_val_acc = None
        self.add_v_number = add_v_number

    def on_epoch_end(self, trainer, pl_module):
        if not self.best_val_acc or pl_module.val_results_history[-1]["val_acc"] > self.best_val_acc:
            print("new best val acc", pl_module.val_results_history[-1]["val_acc"])
            self.best_val_acc = pl_module.val_results_history[-1]["val_acc"]
            #save_path = self.model_name + (( "_v{}".format(trainer.logger.version) +  "_best.p") if self.add_v_number else "")
            save_path = self.model_name + str(pl_module.val_results_history[-1]["val_acc"]) + ".p"
            pl_module.save(save_path, overwrite_if_exists=True)
            print("Saved checkpoint at epoch {} at \"{}\"".format((trainer.current_epoch + 1), save_path))

cb = SaveBestCallback("optuna_deep8k_", add_v_number = False)

### Objective

In [10]:
def objective(trial):
    metrics_callback = MetricsCallback()  

    # here we sample the hyper params, similar as in our old random search
    trial_hparams = {"batch_size": 32,
                     "learning_rate": trial.suggest_loguniform("learning_rate", 1e-6, 1e-1),
                     "p_drop": trial.suggest_float("p_drop", 0, 1),
                     "lr_decay": trial.suggest_float("lr_decay", 0.7, 1),
                     "n_hidden": trial.suggest_int("n_hidden", 10, 1000),
                     "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-1)
                    }    

    model = DeepRecursiveCNN8kPLModule(trial_hparams)
    
    # create a trainer
    trainer = pl.Trainer(
        logger=pl.loggers.TensorBoardLogger(config.LOG_DIR, name=type(model.model).__name__),
        max_epochs=20,                                                               
        gpus=1 if torch.cuda.is_available() else None,
        callbacks=[metrics_callback, cb],                                                   # save latest accuracy
        early_stop_callback=PyTorchLightningPruningCallback(trial, monitor="validation_acc"), # early stopping
    )
    
    datasetHandler.load(model, 'training')
    datasetHandler.load(model, 'validation')
    trainer.fit(model)

    # save model
    model.save("saved_modelsq/"+'{}.p'.format(trial.number))

    # return validation accuracy from latest model, as that's what we want to minimize by our hyper param search
    return metrics_callback.metrics[-1]["validation_acc"]

pruner = optuna.pruners.MedianPruner()
study = optuna.create_study(direction="maximize", pruner=pruner, study_name='Deep_8K', storage='sqlite:///Deep_8K_study.db', load_if_exists=True)

[I 2020-07-16 11:52:23,424] Using an existing study with name 'Deep_8K' instead of creating a new one.


### Run Search

In [11]:
study.optimize(objective, n_trials=2000, timeout=1.5*21600) #9h

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

   | Name        | Type               | Params
-----------------------------------------------
0  | model       | DeepRecursiveCNN8k | 2 M   
1  | model.bn0   | BatchNorm1d        | 2     
2  | model.conv1 | Conv1d             | 10 K  
3  | model.bn1   | BatchNorm1d        | 256   
4  | model.pool1 | MaxPool1d          | 0     
5  | model.drop1 | Dropout            | 0     
6  | model.conv2 | Conv1d             | 98 K  
7  | model.bn2   | BatchNorm1d        | 512   
8  | model.pool2 | MaxPool1d          | 0     
9  | model.conv3 | Conv1d             | 393 K 
10 | model.bn3   | BatchNorm1d        | 1 K   
11 | model.pool3 | MaxPool1d          | 0     
12 | model.conv4 | Conv1d             | 1 M   
13 | model.bn4   | BatchNorm1d        | 2 K   
14 | model.pool4 | MaxPool1d          | 0     
15 | model.fc1   | Linear             | 686 K 
16 | model.

Val-Acc=0.01956135151155898


Val-Acc=0.8032009484291642
Train-Acc=0.7464426877470356


Val-Acc=0.7812685240071132
Train-Acc=0.7885375494071146


Val-Acc=0.7889745109662122
Train-Acc=0.8122529644268774


...

Val-Acc=0.7925311203319502
Train-Acc=0.8565217391304348


Detected KeyboardInterrupt, attempting graceful shutdown...



Saved model to "saved_modelsq/123.p"


Exception ignored in: <function tqdm.__del__ at 0x7f7dd3eff550>
Traceback (most recent call last):
  File "/nfs/students/summer-term-2020/project-4/yan/.conda/envs/cnn_project/lib/python3.8/site-packages/tqdm/std.py", line 1085, in __del__
    def __del__(self):
KeyboardInterrupt: 
[I 2020-07-16 18:05:11,529] Finished trial#123 with value: 0.7925311203319502 with parameters: {'learning_rate': 0.00040159915909726134, 'p_drop': 0.5346297520122059, 'lr_decay': 0.8686005568926535, 'n_hidden': 957, 'weight_decay': 0.01206898189891947}. Best is trial#49 with value: 0.8725548310610551.
GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

   | Name        | Type               | Params
-----------------------------------------------
0  | model       | DeepRecursiveCNN8k | 2 M   
1  | model.bn0   | BatchNorm1d        | 2     
2  | model.conv1 | Conv1d             | 10 K  
3  | model.bn1   | BatchNorm1d      

KeyboardInterrupt: 

In [ ]:
#!mkdir saved_modelsq

In [12]:
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
best_trial = study.best_trial
print("  Value: {}".format(best_trial.value))
print("  Params: ")
for key, value in best_trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 125
Best trial:
  Value: 0.8725548310610551
  Params: 
    learning_rate: 0.006121431260436952
    lr_decay: 0.82451476898958
    n_hidden: 538
    p_drop: 0.49459000533756575
    weight_decay: 0.00029942805404168955
